# Causality in multivariate time-series

## case a) test relationship between X_j and Y_k    

### generate data (synthetic datasets)

In [1]:
import math
import numpy as np
import pandas as pd
import random


total_length=1000

def sigmoid(x):
        return 1 / (1 + math.exp(-x))
    
def generate(sig_to_noise): 
    np.random.seed(0)
    z = [1,1,1,1]
    x_arr = np.full((10,4),1.0)
    y_arr = np.full((10,4),3.0)
    x_arr_unini = np.full((10,total_length),0.0)
    y_arr_unini = np.full((10,total_length),0.0)
    x = np.hstack((x_arr, x_arr_unini))
    y = np.hstack((y_arr, y_arr_unini)) 
    
    p = []
    n = [3,3,3,3]
    t1 = [3,3,3,3]
    
    
    for i in range(4,total_length+4):  #time
        z.append(math.tanh(z[i-1]+np.random.normal(0,0.01)))
        p.append(z[i]**2+np.random.normal(0,0.05))
        
        for j in range(0,10):
            m = random.randint(1,5)  #random lag
            n = random.randint(1,5)  #random lag
            x[j][i] = sigmoid(z[i-m])+np.random.normal(0,0.01)
            term1 = sigmoid(z[i-m])
            term2 = sigmoid(x[j][i-n])
            noise = np.random.normal(0,1)
            alpha = (abs(term1+term2)/sig_to_noise)/abs(noise)
            
            y[j][i] = term1+term2+alpha*noise


    x=x[:,-total_length:]    
    y=y[:,-total_length:]
    p=p[-total_length:]
    z=z[-total_length:]
    
    #table format
    df1=pd.DataFrame({"x1":x[0,:]})
    for m in range(1,10):
        df1.insert(loc=len(df1.columns), column='x'+str(m+1), value=x[m,:])
    for m in range(0,10):
        df1.insert(loc=len(df1.columns), column='y'+str(m+1), value=y[m,:])
    df1.insert(loc=len(df1.columns), column='p', value=p)
    df1.insert(loc=len(df1.columns), column='z', value=z)
    return df1

df1 = generate(10)
df1

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y3,y4,y5,y6,y7,y8,y9,y10,p,z
0,0.693071,0.749734,0.740559,0.730026,0.732499,0.738669,0.735497,0.745999,0.734189,0.705529,...,1.315905,1.608329,1.608329,1.608329,1.608329,1.107953,1.315905,1.608329,0.611221,0.768904
1,0.753756,0.683741,0.746386,0.732608,0.722181,0.727579,0.743361,0.727185,0.672798,0.713996,...,1.608329,1.608329,1.265771,1.608329,1.608329,1.315905,1.272908,1.608329,0.387080,0.651297
2,0.718531,0.714920,0.722104,0.725951,0.683002,0.731724,0.724715,0.650578,0.722927,0.732833,...,1.550394,1.315905,1.555777,1.545866,1.315905,1.249525,1.315905,1.262108,0.301979,0.569107
3,0.674211,0.738349,0.742453,0.661326,0.722351,0.680168,0.645651,0.643214,0.745941,0.669091,...,1.267359,1.249525,1.315905,1.555777,1.464042,1.232654,1.608329,1.194044,0.275765,0.502619
4,0.619043,0.733141,0.626638,0.638662,0.732328,0.657389,0.670579,0.611343,0.634421,0.750288,...,1.425759,1.445328,1.608329,1.182366,1.495224,1.406410,1.182853,1.540518,0.260415,0.455732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.526421,0.516246,0.517053,0.521333,0.497070,0.520755,0.495462,0.519600,0.532664,0.523518,...,1.258561,1.025938,1.252965,1.256274,1.028352,1.028842,1.258007,1.026938,-0.022146,0.074694
996,0.518454,0.515656,0.497186,0.519268,0.526580,0.520441,0.521268,0.518023,0.516870,0.522316,...,1.258925,1.029515,1.026879,1.256274,1.261713,1.261894,1.255684,1.033487,-0.006050,0.082098
997,0.499914,0.505554,0.523240,0.510995,0.532502,0.510356,0.507178,0.534066,0.503130,0.526591,...,1.030268,1.260019,1.034851,1.259580,1.028929,1.263254,1.026763,1.258312,0.188805,0.083666
998,0.509237,0.515571,0.519724,0.516923,0.530387,0.538423,0.512905,0.497650,0.523117,0.535129,...,1.031293,1.262669,1.033099,1.031215,1.033162,1.264005,1.031375,1.261298,0.079071,0.081430


In [2]:
df1.to_csv("case_a.csv")

### method 1

In [ ]:
import torch
import os
from torch import optim
import pandas as pd
import random

import gruvae
import train
from data_loader import create_inout_sequences
from functions import t_test
from generate_data_a import generate


data = pd.read_csv("case_a.csv")[:1000]

k=random.randint(0, 9)
l=random.randint(0, 9)

x = data['x'+str(k+1)].tolist()
y = data['y'+str(l+1)].tolist()
z = data["z"].tolist()
p = data["p"].tolist()


  
def obtain_errors(model,trials, train_loader,\
                  val_loader, test_loader, scaler, lam=0,\
                  model_type="full", res="No"):
    torch.manual_seed(0)
    mses = []
    mses_nox = []
    vae_trainer = train.AutoEncoderTrainer(model, optim.Adam,\
                    train_loader, val_loader,\
                    test_loader, scaler, 0.001, lam, model_type)
    for i in range(trials):
        _,mse,mse_nox,_ = vae_trainer.train_val_test_iter(test_loader,\
                                                          "test", res)
        mses.append(mse)
        mses_nox.append(mse_nox)
    return mses,mses_nox


def run_model(z, p, x, y, seq=20):
    torch.manual_seed(0)
    model=gruvae.GRUVAE(1,1,5,5,5,5,2,5,10,3,0.3)
    train_loader, val_loader, test_loader, scaler\
        = create_inout_sequences(z, p, x, y, seq, 800, 100, 100, 10)
    vae_trainer = train.AutoEncoderTrainer(model, optim.Adam,\
                            train_loader, val_loader,\
                            test_loader, scaler, lr=0.001, lam=0.01)
    _,_,model = vae_trainer.train_and_evaluate(50)
    return train_loader, val_loader, test_loader, model, scaler


train_loader, val_loader, test_loader, model, scaler = run_model(z, p, x, y)
full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
                          val_loader, test_loader, scaler)  
        
print(t_test([i.tolist() for i in full_mses],\
             [i.tolist() for i in full_mses_res],alternative='greater'))
             
def calc_pvalues(sig_to_noise_low="default", sig_to_noise_upp="default"):
    
    if sig_to_noise_low !="default":
        data_low = generate(sig_to_noise_low)
        z_low = data_low["z"].tolist()
        p_low = data_low["p"].tolist()
        x_low = data_low["x"].tolist()
        y_low = data_low["y"].tolist()
        
        train_loader, val_loader, test_loader, model, scaler\
            = run_model(z_low, p_low, x_low, y_low)
        full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
            val_loader, test_loader, scaler)  
        p_value_low = t_test([i.tolist() for i in full_mses],\
            [i.tolist() for i in full_mses_res],alternative='less')
    
    if sig_to_noise_upp !="default":
        data_upp = generate(sig_to_noise_upp)
        z_upp = data_upp["z"].tolist()
        p_upp = data_upp["p"].tolist()
        x_upp = data_upp["x"].tolist()
        y_upp = data_upp["y"].tolist()
   
        train_loader, val_loader, test_loader, model, scaler\
            = run_model(z_upp, p_upp, x_upp, y_upp)
        full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
            val_loader, test_loader, scaler)  
        p_value_upp = t_test([i.tolist() for i in full_mses],\
            [i.tolist() for i in full_mses_res],alternative='less')
    
    if  sig_to_noise_low!="default" and sig_to_noise_upp!="default":   
        return p_value_low, p_value_upp
    elif sig_to_noise_low!="default" and sig_to_noise_upp=="default":
        return p_value_low
    elif sig_to_noise_upp!="default" and sig_to_noise_low=="default":
        return p_value_upp
    else:
        pass

    
def vary_alpha(trials, sig_to_noise_low, sig_to_noise_upp, type="bisection",\
               step=0.1):
    p_value_low, p_value_upp = calc_pvalues(sig_to_noise_low,sig_to_noise_upp)    
    print(p_value_low, p_value_upp)
    
    if p_value_low<0.05 and p_value_upp<0.05:
        print("enter new bounds")
        return None
    elif p_value_low>0.05 and p_value_upp>0.05:
        print("enter new bounds")
        return None
    c=0
    if type=="bisection":
        prev_low = 0
        prev_p = 0
        for i in range(trials):
            print(f"\tsig_to_noise_low: {sig_to_noise_low}, p_low: {p_value_low}, sig_to_noise_upp: {sig_to_noise_upp}, p_upp: {p_value_upp}")            
            prev_low = sig_to_noise_low
            prev_p = p_value_low
            if p_value_low>0.05:
                sig_to_noise_low = (sig_to_noise_upp+sig_to_noise_low)/2
                p_value_low = calc_pvalues(sig_to_noise_low=sig_to_noise_low)
            if p_value_low<0.05:
                sig_to_noise_upp = sig_to_noise_low
                sig_to_noise_low = prev_low
                p_value_upp = p_value_low
                p_value_low = prev_p
    else:
        while c==0:
            sig_to_noise_low+=step
            p_value_low = calc_pvalues(sig_to_noise_low=sig_to_noise_low)
            print(f"\tsig_to_noise_low: {sig_to_noise_low}, p_low: {p_value_low}, sig_to_noise_upp: {sig_to_noise_upp}, p_upp: {p_value_upp}")
            if p_value_low<0.05:       
                break


def vary_seq(seqs, z, p, x, y):
    for i in seqs:
        train_loader, val_loader, test_loader, model, scaler\
            = run_model(z, p, x, y, i)
        full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
                          val_loader, test_loader, scaler) 
        p_val = t_test([i.tolist() for i in full_mses],\
             [i.tolist() for i in full_mses_res],alternative='less')  
        print(f"\tlength: {i}, p_val: {p_val}")

'''
if __name__=="__main__":
    #vary_alpha(1,5, type="e", step=1)
    #vary_alpha(3,4, type="e", step=0.1)
    #vary_alpha(3,3.1, type="e", step=0.01) #3.05
    
    #vary_alpha(8,0.5,10)
    vary_seq([4,6,8,10,12,14,16], z, p, x, y)
'''   

### method 2

In [ ]:
import torch
import os
from torch import optim
import pandas as pd
import random

import gruvae
import train
from data_loader import create_inout_sequences
from functions import t_test
from generate_data_a import generate


data = pd.read_csv("case_a.csv")[:1000]

k=random.randint(0, 9)
l=random.randint(0, 9)

x = data['x'+str(k+1)].tolist()
y = data['y'+str(l+1)].tolist()
z = data["z"].tolist()

data = data.iloc[: , 1:]
p1 = data[data.columns.difference(['x'+str(k+1), 'y'+str(l+1), "z"])].values.tolist()


  
def obtain_errors(model,trials, train_loader,\
                  val_loader, test_loader, scaler, lam=0,\
                  model_type="full", res="No"):
    torch.manual_seed(0)
    mses = []
    mses_nox = []
    vae_trainer = train.AutoEncoderTrainer(model, optim.Adam,\
                    train_loader, val_loader,\
                    test_loader, scaler, 0.001, lam, model_type)
    for i in range(trials):
        _,mse,mse_nox,_ = vae_trainer.train_val_test_iter(test_loader,\
                                                          "test", res)
        mses.append(mse)
        mses_nox.append(mse_nox)
    return mses,mses_nox


def run_model(z, p1, x, y, seq=20):
    torch.manual_seed(0)
    model=gruvae.GRUVAE(1,1,5,5,5,5,2,5,10,3,0.3)
    train_loader, val_loader, test_loader, scaler\
        = create_inout_sequences(z, p1, x, y, seq, 800, 100, 100, 10)
    vae_trainer = train.AutoEncoderTrainer(model, optim.Adam,\
                            train_loader, val_loader,\
                            test_loader, scaler, lr=0.001, lam=0.01)
    _,_,model = vae_trainer.train_and_evaluate(50)
    return train_loader, val_loader, test_loader, model, scaler


train_loader, val_loader, test_loader, model, scaler = run_model(z, p1, x, y)
full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
                          val_loader, test_loader, scaler)  
        
print(t_test([i.tolist() for i in full_mses],\
             [i.tolist() for i in full_mses_res],alternative='greater'))
             
def calc_pvalues(sig_to_noise_low="default", sig_to_noise_upp="default"):
    
    if sig_to_noise_low !="default":
        data_low = generate(sig_to_noise_low)
        z_low = data_low["z"].tolist()
        p_low = data_low["p1"].tolist()
        x_low = data_low["x"].tolist()
        y_low = data_low["y"].tolist()
        
        train_loader, val_loader, test_loader, model, scaler\
            = run_model(z_low, p_low, x_low, y_low)
        full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
            val_loader, test_loader, scaler)  
        p_value_low = t_test([i.tolist() for i in full_mses],\
            [i.tolist() for i in full_mses_res],alternative='less')
    
    if sig_to_noise_upp !="default":
        data_upp = generate(sig_to_noise_upp)
        z_upp = data_upp["z"].tolist()
        p_upp = data_upp["p1"].tolist()
        x_upp = data_upp["x"].tolist()
        y_upp = data_upp["y"].tolist()
   
        train_loader, val_loader, test_loader, model, scaler\
            = run_model(z_upp, p_upp, x_upp, y_upp)
        full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
            val_loader, test_loader, scaler)  
        p_value_upp = t_test([i.tolist() for i in full_mses],\
            [i.tolist() for i in full_mses_res],alternative='less')
    
    if  sig_to_noise_low!="default" and sig_to_noise_upp!="default":   
        return p_value_low, p_value_upp
    elif sig_to_noise_low!="default" and sig_to_noise_upp=="default":
        return p_value_low
    elif sig_to_noise_upp!="default" and sig_to_noise_low=="default":
        return p_value_upp
    else:
        pass

    
def vary_alpha(trials, sig_to_noise_low, sig_to_noise_upp, type="bisection",\
               step=0.1):
    p_value_low, p_value_upp = calc_pvalues(sig_to_noise_low,sig_to_noise_upp)    
    print(p_value_low, p_value_upp)
    
    if p_value_low<0.05 and p_value_upp<0.05:
        print("enter new bounds")
        return None
    elif p_value_low>0.05 and p_value_upp>0.05:
        print("enter new bounds")
        return None
    c=0
    if type=="bisection":
        prev_low = 0
        prev_p = 0
        for i in range(trials):
            print(f"\tsig_to_noise_low: {sig_to_noise_low}, p_low: {p_value_low}, sig_to_noise_upp: {sig_to_noise_upp}, p_upp: {p_value_upp}")            
            prev_low = sig_to_noise_low
            prev_p = p_value_low
            if p_value_low>0.05:
                sig_to_noise_low = (sig_to_noise_upp+sig_to_noise_low)/2
                p_value_low = calc_pvalues(sig_to_noise_low=sig_to_noise_low)
            if p_value_low<0.05:
                sig_to_noise_upp = sig_to_noise_low
                sig_to_noise_low = prev_low
                p_value_upp = p_value_low
                p_value_low = prev_p
    else:
        while c==0:
            sig_to_noise_low+=step
            p_value_low = calc_pvalues(sig_to_noise_low=sig_to_noise_low)
            print(f"\tsig_to_noise_low: {sig_to_noise_low}, p_low: {p_value_low}, sig_to_noise_upp: {sig_to_noise_upp}, p_upp: {p_value_upp}")
            if p_value_low<0.05:       
                break


def vary_seq(seqs, z, p1, x, y):
    for i in seqs:
        train_loader, val_loader, test_loader, model, scaler\
            = run_model(z, p1, x, y, i)
        full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
                          val_loader, test_loader, scaler) 
        p_val = t_test([i.tolist() for i in full_mses],\
             [i.tolist() for i in full_mses_res],alternative='less')  
        print(f"\tlength: {i}, p_val: {p_val}")

'''
if __name__=="__main__":
    #vary_alpha(1,5, type="e", step=1)
    #vary_alpha(3,4, type="e", step=0.1)
    #vary_alpha(3,3.1, type="e", step=0.01) #3.05
    
    #vary_alpha(8,0.5,10)
    vary_seq([4,6,8,10,12,14,16], z, p1, x, y)
'''   

### method 3

In [ ]:
import torch
import os
from torch import optim
import pandas as pd
import random

import pandas as pd  # to load the dataframe
from sklearn.preprocessing import StandardScaler  # to standardize the features
from sklearn.decomposition import PCA  # to apply PCA

import gruvae
import train
from data_loader import create_inout_sequences
from functions import t_test
from generate_data_a import generate


data = pd.read_csv("case_a.csv")[:1000]

k=random.randint(0, 9)
l=random.randint(0, 9)

x = data['x'+str(k+1)].tolist()
y = data['y'+str(l+1)].tolist()
z = data["z"].tolist()

data = data.iloc[: , 1:]
p1 = data[data.columns.difference(['x'+str(k+1), 'y'+str(l+1), "z"])]

#PCA
#convert the dataset into a pandas data frame
df = pd.DataFrame(p1)

#Standardize the features
#Create an object of StandardScaler which is present in sklearn.preprocessing
scalar = StandardScaler()
scaled_data = pd.DataFrame(scalar.fit_transform(df), columns=df.columns) #scaling the data

#Applying PCA
#Taking no. of Principal Components as 1
pca = PCA(n_components = 1)
pca.fit(scaled_data)
data_pca = pca.transform(scaled_data)
data_pca = pd.DataFrame(data_pca,columns=['PC1'])
p2 = data_pca['PC1'].tolist()




  
def obtain_errors(model,trials, train_loader,\
                  val_loader, test_loader, scaler, lam=0,\
                  model_type="full", res="No"):
    torch.manual_seed(0)
    mses = []
    mses_nox = []
    vae_trainer = train.AutoEncoderTrainer(model, optim.Adam,\
                    train_loader, val_loader,\
                    test_loader, scaler, 0.001, lam, model_type)
    for i in range(trials):
        _,mse,mse_nox,_ = vae_trainer.train_val_test_iter(test_loader,\
                                                          "test", res)
        mses.append(mse)
        mses_nox.append(mse_nox)
    return mses,mses_nox


def run_model(z, p2, x, y, seq=20):
    torch.manual_seed(0)
    model=gruvae.GRUVAE(1,1,5,5,5,5,2,5,10,3,0.3)
    train_loader, val_loader, test_loader, scaler\
        = create_inout_sequences(z, p2, x, y, seq, 800, 100, 100, 10)
    vae_trainer = train.AutoEncoderTrainer(model, optim.Adam,\
                            train_loader, val_loader,\
                            test_loader, scaler, lr=0.001, lam=0.01)
    _,_,model = vae_trainer.train_and_evaluate(50)
    return train_loader, val_loader, test_loader, model, scaler


train_loader, val_loader, test_loader, model, scaler = run_model(z, p2, x, y)
full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
                          val_loader, test_loader, scaler)  
        
print(t_test([i.tolist() for i in full_mses],\
             [i.tolist() for i in full_mses_res],alternative='greater'))
             
def calc_pvalues(sig_to_noise_low="default", sig_to_noise_upp="default"):
    
    if sig_to_noise_low !="default":
        data_low = generate(sig_to_noise_low)
        z_low = data_low["z"].tolist()
        p_low = data_low["p2"].tolist()
        x_low = data_low["x"].tolist()
        y_low = data_low["y"].tolist()
        
        train_loader, val_loader, test_loader, model, scaler\
            = run_model(z_low, p_low, x_low, y_low)
        full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
            val_loader, test_loader, scaler)  
        p_value_low = t_test([i.tolist() for i in full_mses],\
            [i.tolist() for i in full_mses_res],alternative='less')
    
    if sig_to_noise_upp !="default":
        data_upp = generate(sig_to_noise_upp)
        z_upp = data_upp["z"].tolist()
        p_upp = data_upp["p2"].tolist()
        x_upp = data_upp["x"].tolist()
        y_upp = data_upp["y"].tolist()
   
        train_loader, val_loader, test_loader, model, scaler\
            = run_model(z_upp, p_upp, x_upp, y_upp)
        full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
            val_loader, test_loader, scaler)  
        p_value_upp = t_test([i.tolist() for i in full_mses],\
            [i.tolist() for i in full_mses_res],alternative='less')
    
    if  sig_to_noise_low!="default" and sig_to_noise_upp!="default":   
        return p_value_low, p_value_upp
    elif sig_to_noise_low!="default" and sig_to_noise_upp=="default":
        return p_value_low
    elif sig_to_noise_upp!="default" and sig_to_noise_low=="default":
        return p_value_upp
    else:
        pass

    
def vary_alpha(trials, sig_to_noise_low, sig_to_noise_upp, type="bisection",\
               step=0.1):
    p_value_low, p_value_upp = calc_pvalues(sig_to_noise_low,sig_to_noise_upp)    
    print(p_value_low, p_value_upp)
    
    if p_value_low<0.05 and p_value_upp<0.05:
        print("enter new bounds")
        return None
    elif p_value_low>0.05 and p_value_upp>0.05:
        print("enter new bounds")
        return None
    c=0
    if type=="bisection":
        prev_low = 0
        prev_p = 0
        for i in range(trials):
            print(f"\tsig_to_noise_low: {sig_to_noise_low}, p_low: {p_value_low}, sig_to_noise_upp: {sig_to_noise_upp}, p_upp: {p_value_upp}")            
            prev_low = sig_to_noise_low
            prev_p = p_value_low
            if p_value_low>0.05:
                sig_to_noise_low = (sig_to_noise_upp+sig_to_noise_low)/2
                p_value_low = calc_pvalues(sig_to_noise_low=sig_to_noise_low)
            if p_value_low<0.05:
                sig_to_noise_upp = sig_to_noise_low
                sig_to_noise_low = prev_low
                p_value_upp = p_value_low
                p_value_low = prev_p
    else:
        while c==0:
            sig_to_noise_low+=step
            p_value_low = calc_pvalues(sig_to_noise_low=sig_to_noise_low)
            print(f"\tsig_to_noise_low: {sig_to_noise_low}, p_low: {p_value_low}, sig_to_noise_upp: {sig_to_noise_upp}, p_upp: {p_value_upp}")
            if p_value_low<0.05:       
                break


def vary_seq(seqs, z, p2, x, y):
    for i in seqs:
        train_loader, val_loader, test_loader, model, scaler\
            = run_model(z, p2, x, y, i)
        full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
                          val_loader, test_loader, scaler) 
        p_val = t_test([i.tolist() for i in full_mses],\
             [i.tolist() for i in full_mses_res],alternative='less')  
        print(f"\tlength: {i}, p_val: {p_val}")

'''
if __name__=="__main__":
    #vary_alpha(1,5, type="e", step=1)
    #vary_alpha(3,4, type="e", step=0.1)
    #vary_alpha(3,3.1, type="e", step=0.01) #3.05
    
    #vary_alpha(8,0.5,10)
    vary_seq([4,6,8,10,12,14,16], z, p2, x, y)
'''   

## case b) test relationship between X_j and X_k

### generate data (synthetic datasets)

In [3]:
import math
import numpy as np
import pandas as pd
import random


total_length=1000

def sigmoid(x):
        return 1 / (1 + math.exp(-x))
    
def generate(sig_to_noise): 
    np.random.seed(0)
    z = [1,1,1,1]
    x_arr = np.full((10,4),1.0)
    x_arr_unini = np.full((10,total_length),0.0)
    x = np.hstack((x_arr, x_arr_unini))
    
    p = []
    n = [3,3,3,3]
    t1 = [3,3,3,3]
    
    
    for i in range(4,total_length+4):   #time
        z.append(math.tanh(z[i-1]+np.random.normal(0,0.01)))
        p.append(z[i]**2+np.random.normal(0,0.05))
        
        for j in range(0,10):
            k = random.randint(0,9)
            m = random.randint(1,5)  #random lag
            n = random.randint(1,5)  #random lag
            term1 = sigmoid(z[i-m])
            term2 = sigmoid(x[k][i-n])
            noise = np.random.normal(0,1)
            alpha = (abs(term1+term2)/sig_to_noise)/abs(noise)  
                
            x[j][i] = term1+term2+alpha*noise



    x=x[:,-total_length:]    
    p=p[-total_length:]
    z=z[-total_length:]
    
     #table format
    df2=pd.DataFrame({"x1":x[0,:]})
    for m in range(1,10):
        df2.insert(loc=len(df2.columns), column='x'+str(m+1), value=x[m,:])
    
    
    df2.insert(loc=len(df2.columns), column='p', value=p)
    df2.insert(loc=len(df2.columns), column='z', value=z)
   
    return df2


df2=generate(10)
df2

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,p,z
0,1.608329,1.354164,1.608329,1.315905,1.354164,1.107953,1.315905,1.354164,1.301612,1.354164,0.611221,0.768904
1,1.608329,1.608329,1.671514,1.315905,1.631086,1.315905,1.367602,1.555777,1.608329,1.315905,0.429496,0.650701
2,1.608329,1.315905,1.608329,1.555777,1.608329,1.608329,1.249404,1.249404,1.272908,1.608329,0.272099,0.587211
3,1.357430,1.272908,1.236405,1.341313,1.367602,1.608329,1.328314,1.344138,1.364817,1.581288,0.348170,0.536704
4,1.404401,1.618962,1.364817,1.367602,1.279446,1.521072,1.555777,1.498314,1.328808,1.277930,0.217977,0.478136
...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.396280,1.104147,1.148784,1.105839,1.345006,1.111737,1.103410,1.141217,1.396280,1.405916,0.011240,-0.100488
996,1.348877,1.361154,1.353232,1.405952,1.398258,1.103218,1.141604,1.142386,1.391009,1.348613,-0.066545,-0.101959
997,1.391228,1.137854,1.142048,1.148087,1.360473,1.352925,1.144433,1.399007,1.359718,1.404487,-0.056218,-0.108295
998,1.148466,1.145256,1.140209,1.101261,1.151812,1.404010,1.110562,1.398614,1.397809,1.113050,0.107108,-0.116596


In [4]:
df2.to_csv("case_b.csv")

### method 1

In [ ]:
import torch
import os
from torch import optim
import pandas as pd
import random

import gruvae
import train
from data_loader import create_inout_sequences
from functions import t_test
from generate_data_b import generate



data = pd.read_csv("case_b.csv")[:1000]


m = random.randint(0, 9)
x_j = data['x'+str(m+1)].tolist()

numbers = list(range(0, 10))
numbers.remove(m)
l = random.choice(numbers)
x_k = data['x'+str(l+1)].tolist()

z = data["z"].tolist()
p = data["p"].tolist()


  
def obtain_errors(model,trials, train_loader,\
                  val_loader, test_loader, scaler, lam=0,\
                  model_type="full", res="No"):
    torch.manual_seed(0)
    mses = []
    mses_nox = []
    vae_trainer = train.AutoEncoderTrainer(model, optim.Adam,\
                    train_loader, val_loader,\
                    test_loader, scaler, 0.001, lam, model_type)
    for i in range(trials):
        _,mse,mse_nox,_ = vae_trainer.train_val_test_iter(test_loader,\
                                                          "test", res)
        mses.append(mse)
        mses_nox.append(mse_nox)
    return mses,mses_nox


def run_model(z, p, x_j, x_k, seq=20):
    torch.manual_seed(0)
    model=gruvae.GRUVAE(1,1,5,5,5,5,2,5,10,3,0.3)
    train_loader, val_loader, test_loader, scaler\
        = create_inout_sequences(z, p, x_j, x_k, seq, 800, 100, 100, 10)
    vae_trainer = train.AutoEncoderTrainer(model, optim.Adam,\
                            train_loader, val_loader,\
                            test_loader, scaler, lr=0.001, lam=0.01)
    _,_,model = vae_trainer.train_and_evaluate(50)
    return train_loader, val_loader, test_loader, model, scaler


train_loader, val_loader, test_loader, model, scaler = run_model(z, p, x_j, x_k)
full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
                          val_loader, test_loader, scaler)  
        
print(t_test([i.tolist() for i in full_mses],\
             [i.tolist() for i in full_mses_res],alternative='greater'))
             
def calc_pvalues(sig_to_noise_low="default", sig_to_noise_upp="default"):
    
    if sig_to_noise_low !="default":
        data_low = generate(sig_to_noise_low)
        z_low = data_low["z"].tolist()
        p_low = data_low["p"].tolist()
        x_low = data_low['x'+str(m+1)].tolist()
        y_low = data_low['x'+str(l+1)].tolist()
        
        train_loader, val_loader, test_loader, model, scaler\
            = run_model(z_low, p_low, x_low, y_low)
        full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
            val_loader, test_loader, scaler)  
        p_value_low = t_test([i.tolist() for i in full_mses],\
            [i.tolist() for i in full_mses_res],alternative='less')
    
    if sig_to_noise_upp !="default":
        data_upp = generate(sig_to_noise_upp)
        z_upp = data_upp["z"].tolist()
        p_upp = data_upp["p"].tolist()
        x_upp = data_upp['x'+str(m+1)].tolist()
        y_upp = data_upp['x'+str(l+1)].tolist()
   
        train_loader, val_loader, test_loader, model, scaler\
            = run_model(z_upp, p_upp, x_upp, y_upp)
        full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
            val_loader, test_loader, scaler)  
        p_value_upp = t_test([i.tolist() for i in full_mses],\
            [i.tolist() for i in full_mses_res],alternative='less')
    
    if  sig_to_noise_low!="default" and sig_to_noise_upp!="default":   
        return p_value_low, p_value_upp
    elif sig_to_noise_low!="default" and sig_to_noise_upp=="default":
        return p_value_low
    elif sig_to_noise_upp!="default" and sig_to_noise_low=="default":
        return p_value_upp
    else:
        pass

    
def vary_alpha(trials, sig_to_noise_low, sig_to_noise_upp, type="bisection",\
               step=0.1):
    p_value_low, p_value_upp = calc_pvalues(sig_to_noise_low,sig_to_noise_upp)    
    print(p_value_low, p_value_upp)
    
    if p_value_low<0.05 and p_value_upp<0.05:
        print("enter new bounds")
        return None
    elif p_value_low>0.05 and p_value_upp>0.05:
        print("enter new bounds")
        return None
    c=0
    if type=="bisection":
        prev_low = 0
        prev_p = 0
        for i in range(trials):
            print(f"\tsig_to_noise_low: {sig_to_noise_low}, p_low: {p_value_low}, sig_to_noise_upp: {sig_to_noise_upp}, p_upp: {p_value_upp}")            
            prev_low = sig_to_noise_low
            prev_p = p_value_low
            if p_value_low>0.05:
                sig_to_noise_low = (sig_to_noise_upp+sig_to_noise_low)/2
                p_value_low = calc_pvalues(sig_to_noise_low=sig_to_noise_low)
            if p_value_low<0.05:
                sig_to_noise_upp = sig_to_noise_low
                sig_to_noise_low = prev_low
                p_value_upp = p_value_low
                p_value_low = prev_p
    else:
        while c==0:
            sig_to_noise_low+=step
            p_value_low = calc_pvalues(sig_to_noise_low=sig_to_noise_low)
            print(f"\tsig_to_noise_low: {sig_to_noise_low}, p_low: {p_value_low}, sig_to_noise_upp: {sig_to_noise_upp}, p_upp: {p_value_upp}")
            if p_value_low<0.05:       
                break


def vary_seq(seqs, z, p, x_j, x_k):
    for i in seqs:
        train_loader, val_loader, test_loader, model, scaler\
            = run_model(z, p, x_j, x_k, i)
        full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
                          val_loader, test_loader, scaler) 
        p_val = t_test([i.tolist() for i in full_mses],\
             [i.tolist() for i in full_mses_res],alternative='less')  
        print(f"\tlength: {i}, p_val: {p_val}")

'''
if __name__=="__main__":
    #vary_alpha(1,5, type="e", step=1)
    #vary_alpha(3,4, type="e", step=0.1)
    #vary_alpha(3,3.1, type="e", step=0.01) #3.05
    
    #vary_alpha(8,0.5,10)
    vary_seq([4,6,8,10,12,14,16], z, p, x_j, x_k)
'''   

### method 2

In [ ]:
import torch
import os
from torch import optim
import pandas as pd
import random

import gruvae
import train
from data_loader import create_inout_sequences
from functions import t_test
from generate_data_b import generate


data = pd.read_csv("case_b.csv")[:1000]


z = data["z"].tolist()

m = random.randint(0, 9)
x_j = data['x'+str(m+1)].tolist()

numbers = list(range(0, 10))
numbers.remove(m)
l = random.choice(numbers)
x_k = data['x'+str(l+1)].tolist()

data = data.iloc[: , 1:]
p1 = data[data.columns.difference(['x'+str(m+1), 'y'+str(l+1), "z"])].values.tolist()


  
def obtain_errors(model,trials, train_loader,\
                  val_loader, test_loader, scaler, lam=0,\
                  model_type="full", res="No"):
    torch.manual_seed(0)
    mses = []
    mses_nox = []
    vae_trainer = train.AutoEncoderTrainer(model, optim.Adam,\
                    train_loader, val_loader,\
                    test_loader, scaler, 0.001, lam, model_type)
    for i in range(trials):
        _,mse,mse_nox,_ = vae_trainer.train_val_test_iter(test_loader,\
                                                          "test", res)
        mses.append(mse)
        mses_nox.append(mse_nox)
    return mses,mses_nox


def run_model(z, p1, x_j, x_k, seq=20):
    torch.manual_seed(0)
    model=gruvae.GRUVAE(1,1,5,5,5,5,2,5,10,3,0.3)
    train_loader, val_loader, test_loader, scaler\
        = create_inout_sequences(z, p1, x_i, x_j, seq, 800, 100, 100, 10)
    vae_trainer = train.AutoEncoderTrainer(model, optim.Adam,\
                            train_loader, val_loader,\
                            test_loader, scaler, lr=0.001, lam=0.01)
    _,_,model = vae_trainer.train_and_evaluate(50)
    return train_loader, val_loader, test_loader, model, scaler


train_loader, val_loader, test_loader, model, scaler = run_model(z, p1, x_j, x_k)
full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
                          val_loader, test_loader, scaler)  
        
print(t_test([i.tolist() for i in full_mses],\
             [i.tolist() for i in full_mses_res],alternative='greater'))
             
def calc_pvalues(sig_to_noise_low="default", sig_to_noise_upp="default"):
    
    if sig_to_noise_low !="default":
        data_low = generate(sig_to_noise_low)
        z_low = data_low["z"].tolist()
        p_low = data_low["p1"].tolist()
        x_low = data_low['x'+str(m+1)].tolist()
        y_low = data_low['x'+str(l+1)].tolist()
        
        train_loader, val_loader, test_loader, model, scaler\
            = run_model(z_low, p_low, x_low, y_low)
        full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
            val_loader, test_loader, scaler)  
        p_value_low = t_test([i.tolist() for i in full_mses],\
            [i.tolist() for i in full_mses_res],alternative='less')
    
    if sig_to_noise_upp !="default":
        data_upp = generate(sig_to_noise_upp)
        z_upp = data_upp["z"].tolist()
        p_upp = data_upp["p1"].tolist()
        x_upp = data_upp['x'+str(m+1)].tolist()
        y_upp = data_upp['x'+str(l+1)].tolist()
   
        train_loader, val_loader, test_loader, model, scaler\
            = run_model(z_upp, p_upp, x_upp, y_upp)
        full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
            val_loader, test_loader, scaler)  
        p_value_upp = t_test([i.tolist() for i in full_mses],\
            [i.tolist() for i in full_mses_res],alternative='less')
    
    if  sig_to_noise_low!="default" and sig_to_noise_upp!="default":   
        return p_value_low, p_value_upp
    elif sig_to_noise_low!="default" and sig_to_noise_upp=="default":
        return p_value_low
    elif sig_to_noise_upp!="default" and sig_to_noise_low=="default":
        return p_value_upp
    else:
        pass

    
def vary_alpha(trials, sig_to_noise_low, sig_to_noise_upp, type="bisection",\
               step=0.1):
    p_value_low, p_value_upp = calc_pvalues(sig_to_noise_low,sig_to_noise_upp)    
    print(p_value_low, p_value_upp)
    
    if p_value_low<0.05 and p_value_upp<0.05:
        print("enter new bounds")
        return None
    elif p_value_low>0.05 and p_value_upp>0.05:
        print("enter new bounds")
        return None
    c=0
    if type=="bisection":
        prev_low = 0
        prev_p = 0
        for i in range(trials):
            print(f"\tsig_to_noise_low: {sig_to_noise_low}, p_low: {p_value_low}, sig_to_noise_upp: {sig_to_noise_upp}, p_upp: {p_value_upp}")            
            prev_low = sig_to_noise_low
            prev_p = p_value_low
            if p_value_low>0.05:
                sig_to_noise_low = (sig_to_noise_upp+sig_to_noise_low)/2
                p_value_low = calc_pvalues(sig_to_noise_low=sig_to_noise_low)
            if p_value_low<0.05:
                sig_to_noise_upp = sig_to_noise_low
                sig_to_noise_low = prev_low
                p_value_upp = p_value_low
                p_value_low = prev_p
    else:
        while c==0:
            sig_to_noise_low+=step
            p_value_low = calc_pvalues(sig_to_noise_low=sig_to_noise_low)
            print(f"\tsig_to_noise_low: {sig_to_noise_low}, p_low: {p_value_low}, sig_to_noise_upp: {sig_to_noise_upp}, p_upp: {p_value_upp}")
            if p_value_low<0.05:       
                break


def vary_seq(seqs, z, p1, x_j, x_k):
    for i in seqs:
        train_loader, val_loader, test_loader, model, scaler\
            = run_model(z, p1, x_j, x_k, i)
        full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
                          val_loader, test_loader, scaler) 
        p_val = t_test([i.tolist() for i in full_mses],\
             [i.tolist() for i in full_mses_res],alternative='less')  
        print(f"\tlength: {i}, p_val: {p_val}")

'''
if __name__=="__main__":
    #vary_alpha(1,5, type="e", step=1)
    #vary_alpha(3,4, type="e", step=0.1)
    #vary_alpha(3,3.1, type="e", step=0.01) #3.05
    
    #vary_alpha(8,0.5,10)
    vary_seq([4,6,8,10,12,14,16], z, p1, x_j, x_k)
'''   

### method 3

In [ ]:
import torch
import os
from torch import optim
import pandas as pd
import random

import pandas as pd  # to load the dataframe
from sklearn.preprocessing import StandardScaler  # to standardize the features
from sklearn.decomposition import PCA  # to apply PCA

import gruvae
import train
from data_loader import create_inout_sequences
from functions import t_test
from generate_data_b import generate



data = pd.read_csv("case_b.csv")[:1000]


m = random.randint(0, 9)
x_j = data['x'+str(m+1)].tolist()

numbers = list(range(0, 10))
numbers.remove(m)
l = random.choice(numbers)
x_k = data['x'+str(l+1)].tolist()

z = data["z"].tolist()

data = data.iloc[: , 1:]
p1 = data[data.columns.difference(['x'+str(m+1), 'x'+str(l+1), "z"])]

#PCA

#convert the dataset into a pandas data frame
df = pd.DataFrame(p1)

#Standardize the features
#Create an object of StandardScaler which is present in sklearn.preprocessing
scalar = StandardScaler()
scaled_data = pd.DataFrame(scalar.fit_transform(df), columns=df.columns) #scaling the data
 
#Applying PCA
#Taking no. of Principal Components as 1
pca = PCA(n_components = 1)
pca.fit(scaled_data)
data_pca = pca.transform(scaled_data)
data_pca = pd.DataFrame(data_pca,columns=['PC1'])
p2 = data_pca['PC1'].tolist()






  
def obtain_errors(model,trials, train_loader,\
                  val_loader, test_loader, scaler, lam=0,\
                  model_type="full", res="No"):
    torch.manual_seed(0)
    mses = []
    mses_nox = []
    vae_trainer = train.AutoEncoderTrainer(model, optim.Adam,\
                    train_loader, val_loader,\
                    test_loader, scaler, 0.001, lam, model_type)
    for i in range(trials):
        _,mse,mse_nox,_ = vae_trainer.train_val_test_iter(test_loader,\
                                                          "test", res)
        mses.append(mse)
        mses_nox.append(mse_nox)
    return mses,mses_nox


def run_model(z, p2, x_j, x_k, seq=20):
    torch.manual_seed(0)
    model=gruvae.GRUVAE(1,1,5,5,5,5,2,5,10,3,0.3)
    train_loader, val_loader, test_loader, scaler\
        = create_inout_sequences(z, p2, x_j, x_k, seq, 800, 100, 100, 10)
    vae_trainer = train.AutoEncoderTrainer(model, optim.Adam,\
                            train_loader, val_loader,\
                            test_loader, scaler, lr=0.001, lam=0.01)
    _,_,model = vae_trainer.train_and_evaluate(50)
    return train_loader, val_loader, test_loader, model, scaler


train_loader, val_loader, test_loader, model, scaler = run_model(z, p2, x_j, x_k)
full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
                          val_loader, test_loader, scaler)  
        
print(t_test([i.tolist() for i in full_mses],\
             [i.tolist() for i in full_mses_res],alternative='greater'))
             
def calc_pvalues(sig_to_noise_low="default", sig_to_noise_upp="default"):
    
    if sig_to_noise_low !="default":
        data_low = generate(sig_to_noise_low)
        z_low = data_low["z"].tolist()
        p_low = data_low["p2"].tolist()
        x_low = data_low['x'+str(m+1)].tolist()
        y_low = data_low['x'+str(l+1)].tolist()
        
        train_loader, val_loader, test_loader, model, scaler\
            = run_model(z_low, p_low, x_low, y_low)
        full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
            val_loader, test_loader, scaler)  
        p_value_low = t_test([i.tolist() for i in full_mses],\
            [i.tolist() for i in full_mses_res],alternative='less')
    
    if sig_to_noise_upp !="default":
        data_upp = generate(sig_to_noise_upp)
        z_upp = data_upp["z"].tolist()
        p_upp = data_upp["p2"].tolist()
        x_upp = data_upp['x'+str(m+1)].tolist()
        y_upp = data_upp['x'+str(l+1)].tolist()
   
        train_loader, val_loader, test_loader, model, scaler\
            = run_model(z_upp, p_upp, x_upp, y_upp)
        full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
            val_loader, test_loader, scaler)  
        p_value_upp = t_test([i.tolist() for i in full_mses],\
            [i.tolist() for i in full_mses_res],alternative='less')
    
    if  sig_to_noise_low!="default" and sig_to_noise_upp!="default":   
        return p_value_low, p_value_upp
    elif sig_to_noise_low!="default" and sig_to_noise_upp=="default":
        return p_value_low
    elif sig_to_noise_upp!="default" and sig_to_noise_low=="default":
        return p_value_upp
    else:
        pass

    
def vary_alpha(trials, sig_to_noise_low, sig_to_noise_upp, type="bisection",\
               step=0.1):
    p_value_low, p_value_upp = calc_pvalues(sig_to_noise_low,sig_to_noise_upp)    
    print(p_value_low, p_value_upp)
    
    if p_value_low<0.05 and p_value_upp<0.05:
        print("enter new bounds")
        return None
    elif p_value_low>0.05 and p_value_upp>0.05:
        print("enter new bounds")
        return None
    c=0
    if type=="bisection":
        prev_low = 0
        prev_p = 0
        for i in range(trials):
            print(f"\tsig_to_noise_low: {sig_to_noise_low}, p_low: {p_value_low}, sig_to_noise_upp: {sig_to_noise_upp}, p_upp: {p_value_upp}")            
            prev_low = sig_to_noise_low
            prev_p = p_value_low
            if p_value_low>0.05:
                sig_to_noise_low = (sig_to_noise_upp+sig_to_noise_low)/2
                p_value_low = calc_pvalues(sig_to_noise_low=sig_to_noise_low)
            if p_value_low<0.05:
                sig_to_noise_upp = sig_to_noise_low
                sig_to_noise_low = prev_low
                p_value_upp = p_value_low
                p_value_low = prev_p
    else:
        while c==0:
            sig_to_noise_low+=step
            p_value_low = calc_pvalues(sig_to_noise_low=sig_to_noise_low)
            print(f"\tsig_to_noise_low: {sig_to_noise_low}, p_low: {p_value_low}, sig_to_noise_upp: {sig_to_noise_upp}, p_upp: {p_value_upp}")
            if p_value_low<0.05:       
                break


def vary_seq(seqs, z, p2, x_j, x_k):
    for i in seqs:
        train_loader, val_loader, test_loader, model, scaler\
            = run_model(z, p2, x_j, x_k, i)
        full_mses, full_mses_res = obtain_errors(model, 50, train_loader,\
                          val_loader, test_loader, scaler) 
        p_val = t_test([i.tolist() for i in full_mses],\
             [i.tolist() for i in full_mses_res],alternative='less')  
        print(f"\tlength: {i}, p_val: {p_val}")

'''
if __name__=="__main__":
    #vary_alpha(1,5, type="e", step=1)
    #vary_alpha(3,4, type="e", step=0.1)
    #vary_alpha(3,3.1, type="e", step=0.01) #3.05
    
    #vary_alpha(8,0.5,10)
    vary_seq([4,6,8,10,12,14,16], z, p2, x_j, x_k)
'''   